In [17]:
%pip install -q langchain==0.0.297 \
PyPDF2==3.0.1 \
faiss-cpu==1.7.4 \
google-generativeai==0.1.0 \
InstructorEmbedding==1.0.1 \
sentence_transformers==2.2.2

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests

In [13]:
url = "https://www.uscis.gov/sites/default/files/document/guides/M-654.pdf"
response = requests.get(url, stream=True)

In [14]:
response.headers

{'Server': 'Apache', 'X-Content-Type-Options': 'nosniff', 'Last-Modified': 'Mon, 22 Jun 2020 21:14:25 GMT', 'ETag': '"b5370-5a8b2ba316e40"', 'Accept-Ranges': 'bytes', 'Content-Length': '742256', 'Content-Type': 'application/pdf', 'Cache-Control': 'max-age=35833', 'Expires': 'Thu, 01 Feb 2024 15:14:11 GMT', 'Date': 'Thu, 01 Feb 2024 05:16:58 GMT', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000 ; includeSubDomains ; preload'}

In [15]:
open("M-6541.pdf", "wb").write(response.content)


742256

In [18]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.llms import GooglePalm
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
# from langchain.embeddings import GooglePalmEmbeddings # If no GPU/time

In [20]:
embeddings_model_name = "hkunlp/instructor-xl"
embeddings = HuggingFaceInstructEmbeddings(model_name=embeddings_model_name,model_kwargs={"device": "cuda"})
# Cloud based embedding alternative:
# embeddings = GooglePalmEmbeddings()

C:\Users\yogen\anaconda3\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


C:\Users\yogen\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


max_seq_length  512


In [21]:
pdf = "M-6541.pdf"

if pdf:
    # Extract text from each page
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200,
        length_function=len
        )
    chunks = text_splitter.split_text(text=text)

    # Embed chunks and store them in a Vector Store
    VectorStore = FAISS.from_texts(chunks, embedding=embeddings)

AssertionError: Torch not compiled with CUDA enabled